In [ ]:
def step(self, actions):
    rewards = []
    dones = []
    wins = []
    next_states = []
    agents_reached_target = 0
    
    self.update_grid_colors()
    circle_pos = self.get_circle_grid_position()
    
    reward_position = 0
    reward_bonus = 0
    reward = 0
    done = False
    win = False

    for idx, (agent, action) in enumerate(zip(self.agents, actions)):

        if self.locked[idx]:  # If agent is locked, skip action processing
            next_states.append([self.coords_to_state(agent['coords']), wins[idx], None])  # Use the last known win status
            rewards.append(0)
            dones.append(True)  # Locked agents should be marked as done
            continue

        state = agent['coords']
        base_action = np.array([0, 0])
        message = None
        physical_action = action[0]

        if physical_action == 0:
            base_action[0] = base_action[0]
            base_action[1] = base_action[1]
        elif physical_action == 1:  # up
            if state[1] > UNIT:
                base_action[1] -= UNIT
        elif physical_action == 2:  # down
            if state[1] < (HEIGHT - 1) * UNIT:
                base_action[1] += UNIT
        elif physical_action == 3:  # left
            if state[0] > UNIT:
                base_action[0] -= UNIT
        elif physical_action == 4:  # right
            if state[0] < (WIDTH - 1) * UNIT:
                base_action[0] += UNIT

        initial_pos = self.coords_to_state(state)
        initial_distance = abs(initial_pos[0] - circle_pos[0]) + abs(initial_pos[1] - circle_pos[1])

        self.canvas.move(agent['image_obj'], base_action[0], base_action[1])
        self.canvas.tag_raise(agent['image_obj'])
        next_state = self.canvas.coords(agent['image_obj'])
        
        new_pos = self.coords_to_state(next_state)
        new_distance = abs(new_pos[0] - circle_pos[0]) + abs(new_pos[1] - circle_pos[1])

        reward_position = initial_distance - new_distance

        if new_pos == self.coords_to_state(self.canvas.coords(self.circle)):  # Agent hits the target
                
            agents_reached_target += 1
            if not self.first_agent_reached:
                reward_bonus = 100
                self.first_agent_reached = True
            else:
                reward_bonus = 0

            reward_bonus = 100
            done = False # problem for case base
            win = True
            self.locked[idx] = True #problem for case base
            self.update_grid_colors((0, 0, 255))
                
        elif new_pos in [self.coords_to_state(self.canvas.coords(self.triangle1)), self.coords_to_state(self.canvas.coords(self.triangle2))]:  # Agent hits an obstacle
            reward_bonus = -10
            done = False
            win = False
            self.locked[idx] = True
            self.update_grid_colors((255, 0, 0))
        else:
            reward_bonus = -1
            # done = False
            # win = False
            
        reward = reward_bonus
        # reward = reward_bonus + reward_position

        rewards.append(reward)
        dones.append(done)
        wins.append(win)  # Track the win status for this agent
        
        next_state_obs = self.coords_to_state(next_state)
        next_state_comms = []
        if not self.is_agent_silent:
            for other_agent in self.agents:
                if other_agent == agent:
                    continue

                other_agent_message = actions[other_agent['id']][1]
                next_state_comms.append(other_agent_message)

        next_state_observation = [next_state_obs, win, next_state_comms]

        next_states.append(next_state_observation)

        agent['coords'] = next_state
        
    if all(wins):
        self.update_grid_colors((0, 255, 0))

    if agents_reached_target == self.num_agents and not self.mega_bonus_given:
        for i in range(len(rewards)):
            rewards[i] += 1000  # Mega bonus
        self.mega_bonus_given = True
       
    if all(self.locked):
        dones = [True] * self.num_agents
        self.locked = [False] * self.num_agents

    # self.render()
    return next_states, rewards, dones
